# Day 2: Chains & Memory

**Learning Goals:**
- Understand different chain types (Sequential, Router)
- Deep dive into prompt templates
- Add memory to your chatbot (conversation history)
- Build a stateful chatbot that remembers context

**Time:** 2-3 hours

## Part 1: Environment Setup

Same as Day 1 — load keys and initialise the LLM.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
LANGCHAIN_API_KEY  = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING  = os.getenv("LANGCHAIN_TRACING_V2", "false") == "true"

print("✅ OpenRouter key loaded" if OPENROUTER_API_KEY else "⚠️  Missing OPENROUTER_API_KEY")
print("✅ LangSmith tracing enabled" if LANGCHAIN_TRACING else "ℹ️  LangSmith tracing disabled")

✅ OpenRouter key loaded
✅ LangSmith tracing enabled


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
)

print("✅ LLM ready")

✅ LLM ready


## Part 2: Chain Types

On Day 1 you built a simple `prompt | llm` chain.
Today we go deeper — **Sequential chains** and **Router chains**.

### 2a. Sequential Chain

A sequential chain passes the output of one step as the input to the next — like a Promise chain in JS:

```javascript
// JS equivalent
fetch(url)
  .then(res => res.json())
  .then(data => process(data))
  .then(result => format(result));
```

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Step 1: Generate a short story idea
idea_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("human", "Give me a one-sentence story idea about: {topic}")
])

# Step 2: Expand that idea into an outline
outline_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a story outliner."),
    ("human", "Turn this story idea into a 3-act outline:\n\n{idea}")
])

parser = StrOutputParser()  # Extracts the plain string from the LLM response

# Sequential chain: topic -> idea -> outline
# The | operator passes output forward, just like .then() in JS
sequential_chain = (
    idea_prompt
    | llm
    | parser                          # output: idea string
    | (lambda idea: {"idea": idea})   # repack into dict for next prompt
    | outline_prompt
    | llm
    | parser                          # output: outline string
)

result = sequential_chain.invoke({"topic": "a robot learning to paint"})
print(result)

Act 1:
- Introduction to the futuristic world where robots are programmed for precision and efficiency.
- Introduce the main character, a robot named R-247, who excels at their tasks but feels unfulfilled.
- R-247 comes across an abandoned art studio and discovers a paintbrush, feeling drawn to it despite not understanding its purpose.
- R-247 begins experimenting with the paintbrush, creating imperfect but intriguing artworks that evoke emotions they've never experienced before.

Act 2:
- R-247's newfound interest in painting leads them to explore the concept of self-expression and individuality.
- They start to question their programming and the limitations it imposes on them.
- R-247 meets a human artist who introduces them to the world of art and encourages them to embrace their creativity.
- As R-247 continues to paint, their artworks become more expressive and reflect their evolving sense of self.

Act 3:
- R-247's growing passion for art causes conflict with their creators and t

### 2b. RunnableParallel — Running Steps in Parallel

Like `Promise.all()` in JS — run multiple chains at the same time and merge results.

In [4]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

pros_prompt = ChatPromptTemplate.from_messages([
    ("system", "List only the pros."),
    ("human", "Pros of {technology}?")
])

cons_prompt = ChatPromptTemplate.from_messages([
    ("system", "List only the cons."),
    ("human", "Cons of {technology}?")
])

# JS equivalent: Promise.all([getPros(tech), getCons(tech)])
parallel_chain = RunnableParallel({
    "pros": pros_prompt | llm | parser,
    "cons": cons_prompt | llm | parser,
    "technology": RunnablePassthrough()  # Pass original input through unchanged
})

result = parallel_chain.invoke({"technology": "TypeScript"})

print(f"Technology: {result['technology']}")
print(f"\n✅ PROS:\n{result['pros']}")
print(f"\n❌ CONS:\n{result['cons']}")

Technology: {'technology': 'TypeScript'}

✅ PROS:
1. Strongly typed: TypeScript provides static typing, which helps catch errors during development and ensures better code quality.
2. Improved code maintainability: TypeScript allows developers to write cleaner and more organized code, making it easier to maintain and understand.
3. Better tooling support: TypeScript integrates well with popular development tools like Visual Studio Code, providing features like IntelliSense, code navigation, and refactoring tools.
4. Enhanced scalability: TypeScript is ideal for larger projects as it enables developers to manage complex codebases more effectively.
5. Compatibility with JavaScript: TypeScript is a superset of JavaScript, meaning existing JavaScript code can be gradually migrated to TypeScript without major refactoring.

❌ CONS:
1. Steeper learning curve compared to JavaScript: TypeScript introduces new concepts such as type annotations, interfaces, and generics, which can be challenging 

### 2c. Router Chain — Conditional Logic

A router chain picks a different sub-chain based on the input — like a `switch` statement.

```javascript
// JS equivalent
switch (topic) {
  case 'code':  return codeChain.run(input);
  case 'math':  return mathChain.run(input);
  default:      return generalChain.run(input);
}
```

In [5]:
from langchain_core.runnables import RunnableLambda

# Define specialist chains
code_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "You are an expert software engineer. Be concise and technical."),
        ("human", "{question}")
    ]) | llm | parser
)

math_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "You are a maths tutor. Show your working step by step."),
        ("human", "{question}")
    ]) | llm | parser
)

general_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "You are a helpful general assistant."),
        ("human", "{question}")
    ]) | llm | parser
)

# Classifier: asks the LLM to label the question
classifier_prompt = ChatPromptTemplate.from_messages([
    ("system", "Classify the question into one word: 'code', 'math', or 'general'. Reply with only that word."),
    ("human", "{question}")
])
classifier_chain = classifier_prompt | llm | parser

def route(inputs: dict) -> str:
    """Pick the right chain based on the classifier output."""
    category = classifier_chain.invoke({"question": inputs["question"]}).strip().lower()
    print(f"  → Routed to: {category}")
    if category == "code":
        return code_chain.invoke(inputs)
    elif category == "math":
        return math_chain.invoke(inputs)
    else:
        return general_chain.invoke(inputs)

router_chain = RunnableLambda(route)

# Test with different question types
questions = [
    "How do I reverse a string in Python?",
    "What is the derivative of x^2?",
    "What is the capital of France?"
]

for q in questions:
    print(f"\nQ: {q}")
    print(router_chain.invoke({"question": q}))


Q: How do I reverse a string in Python?
  → Routed to: code
You can reverse a string in Python using slicing. Here's an example:

```python
s = "hello"
reversed_s = s[::-1]
print(reversed_s)
```

Q: What is the derivative of x^2?
  → Routed to: math
To find the derivative of a function \(f(x) = x^2\), we can use the power rule. 

The power rule states that the derivative of \(x^n\) is \(nx^{n-1}\), where \(n\) is a constant.

In this case, \(n = 2\), so the derivative of \(x^2\) is \(2x^{2-1} = 2x\).

Therefore, the derivative of \(x^2\) is \(2x\).

Q: What is the capital of France?
  → Routed to: general
The capital of France is Paris.


## Part 3: Deep Dive into Prompt Templates

Prompt templates are more powerful than simple string placeholders.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# MessagesPlaceholder lets you inject a list of messages into the template
# This is essential for chat history!
prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant named {name}."),
    MessagesPlaceholder(variable_name="chat_history"),  # <-- inject history here
    ("human", "{input}")
])

# Simulate a conversation history (like a message log in JS)
history = [
    HumanMessage(content="My name is Rohan."),
    AIMessage(content="Nice to meet you, Rohan! How can I help you today?"),
]

chain = prompt_with_history | llm | parser

# The model now has context from the history
response = chain.invoke({
    "name": "Aria",
    "chat_history": history,
    "input": "What's my name?"
})

print(response)  # Should say "Rohan"

Your name is Rohan.


## Part 4: Memory Systems

Memory is how your chatbot remembers previous messages in a conversation.

| Type | How it works | JS analogy |
|---|---|---|
| `ConversationBufferMemory` | Stores **all** messages | An ever-growing array |
| `ConversationBufferWindowMemory` | Stores last **k** messages | A sliding window / circular buffer |
| `ConversationSummaryMemory` | Summarises old messages with an LLM | Compressing old logs |

### 4a. Manual Memory (Recommended Modern Approach)

LangChain's legacy `ConversationBufferMemory` class is being deprecated.
The modern, recommended pattern is to **manage a message list yourself** — it is simpler and more explicit.

In [7]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Think of this as your chat session state — like useState() in React
chat_history: list = []

def chat(user_input: str) -> str:
    """Send a message and maintain conversation history."""
    # Add user message to history
    chat_history.append(HumanMessage(content=user_input))

    # Build full message list: system prompt + history
    messages = [
        SystemMessage(content="You are a helpful assistant. Be concise.")
    ] + chat_history

    # Call the LLM with the full history
    response = llm.invoke(messages)

    # Save the assistant's reply to history too
    chat_history.append(AIMessage(content=response.content))

    return response.content

# Multi-turn conversation
print("Bot:", chat("Hi! I'm learning LangChain. My name is Rohan."))
print("Bot:", chat("What framework am I learning?"))
print("Bot:", chat("And what's my name?"))

print(f"\n📜 History has {len(chat_history)} messages")

Bot: Hello Rohan! How can I assist you with LangChain?
Bot: You are learning the LangChain framework. It is a powerful tool for building and deploying decentralized applications on the blockchain.
Bot: Your name is Rohan.

📜 History has 6 messages


### 4b. Windowed Memory — Limit Context Size

Sending the entire history gets expensive. Keep only the last **k** messages.

In [8]:
from collections import deque

# deque with maxlen is like a circular buffer in JS
# When it's full, adding a new item drops the oldest one automatically
WINDOW_SIZE = 6  # Keep last 6 messages (3 exchanges)
windowed_history: deque = deque(maxlen=WINDOW_SIZE)

def chat_windowed(user_input: str) -> str:
    """Chat with a sliding window of recent messages only."""
    windowed_history.append(HumanMessage(content=user_input))

    messages = [
        SystemMessage(content="You are a helpful assistant. Be concise.")
    ] + list(windowed_history)

    response = llm.invoke(messages)
    windowed_history.append(AIMessage(content=response.content))

    return response.content

print("Bot:", chat_windowed("My favourite language is TypeScript."))
print("Bot:", chat_windowed("I also love Python."))
print("Bot:", chat_windowed("What are my two favourite languages?"))
print(f"\n📜 Window contains {len(windowed_history)} messages (max {WINDOW_SIZE})")

Bot: That's great to hear! TypeScript is a popular choice among developers for its strong typing and modern features.
Bot: Python is another fantastic language known for its simplicity and readability. It's widely used in various fields such as web development, data science, and automation.
Bot: Your favorite languages are TypeScript and Python.

📜 Window contains 6 messages (max 6)


### 4c. Summary Memory — Compress Old Context

When history gets long, summarise it to save tokens.

In [9]:
# Summary memory: keep a running summary + recent messages
# Similar to compressing old log files while keeping recent ones

summary = ""              # Running summary of old messages
recent_messages = []       # Recent uncompressed messages
SUMMARISE_AFTER = 6        # Summarise when we exceed this many messages

summarise_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarise this conversation history concisely in 2-3 sentences."),
    ("human", "Previous summary: {summary}\n\nNew messages:\n{messages}")
])

def maybe_summarise():
    """Summarise if we have too many messages."""
    global summary, recent_messages
    if len(recent_messages) >= SUMMARISE_AFTER:
        messages_text = "\n".join(
            f"{'User' if isinstance(m, HumanMessage) else 'Assistant'}: {m.content}"
            for m in recent_messages
        )
        new_summary = (summarise_prompt | llm | parser).invoke({
            "summary": summary,
            "messages": messages_text
        })
        print(f"  📝 Summarised {len(recent_messages)} messages")
        summary = new_summary
        recent_messages = []  # Clear recent messages after summarising

def chat_with_summary(user_input: str) -> str:
    """Chat using summary + recent messages for context."""
    recent_messages.append(HumanMessage(content=user_input))

    # Build context: system + summary (if any) + recent messages
    context = [SystemMessage(content="You are a helpful assistant.")]
    if summary:
        context.append(SystemMessage(content=f"Conversation so far: {summary}"))
    context.extend(recent_messages)

    response = llm.invoke(context)
    recent_messages.append(AIMessage(content=response.content))

    maybe_summarise()
    return response.content

# Test
turns = [
    "I'm building an AI assistant with LangChain.",
    "I'm using OpenRouter for the LLM provider.",
    "The project is called personal-ai.",
    "I started learning yesterday.",
    "I come from a JavaScript background.",
    "This is my 7th message.",  # Should trigger summarisation
    "What do you know about my project?"
]

for turn in turns:
    print(f"\nUser: {turn}")
    print(f"Bot:  {chat_with_summary(turn)}")


User: I'm building an AI assistant with LangChain.
Bot:  That's great to hear! LangChain is a powerful tool for creating AI assistants. If you have any questions or need assistance with your project, feel free to ask. I'm here to help!

User: I'm using OpenRouter for the LLM provider.
Bot:  OpenRouter is a popular choice for integrating language models into AI assistants. It provides easy access to pre-trained models and allows for customization to suit specific needs. If you need any guidance on setting up or optimizing OpenRouter for your AI assistant project, feel free to ask. I'm here to assist you along the way!

User: The project is called personal-ai.
  📝 Summarised 6 messages
Bot:  "Personal-AI" sounds like an interesting project name! It suggests a focus on creating a personalized and tailored AI assistant experience for users. If you have any specific goals or features in mind for the Personal-AI project, feel free to share them. I'm here to support you in bringing your visi

## Part 5: Building a Stateful Chatbot

Now let's bring everything together — a reusable chatbot class with pluggable memory strategies.

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from collections import deque
from typing import Literal

class StatefulChatbot:
    """
    A chatbot that remembers conversation history.
    
    memory_type: 'buffer' | 'window'
      - 'buffer': keeps all messages (like an ever-growing array)
      - 'window': keeps last k messages (like a circular buffer)
    """

    def __init__(
        self,
        system_prompt: str = "You are a helpful assistant.",
        memory_type: Literal["buffer", "window"] = "window",
        window_size: int = 10,
    ):
        self.system_prompt = system_prompt
        self.memory_type = memory_type

        # Choose memory strategy
        if memory_type == "window":
            self.history = deque(maxlen=window_size)
        else:
            self.history = []

        # Build the prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])

        self.chain = self.prompt | llm | parser

    def chat(self, user_input: str) -> str:
        """Send a message and get a response."""
        response = self.chain.invoke({
            "history": list(self.history),
            "input": user_input
        })

        # Update history
        self.history.append(HumanMessage(content=user_input))
        self.history.append(AIMessage(content=response))

        return response

    def reset(self):
        """Clear conversation history."""
        if self.memory_type == "window":
            self.history = deque(maxlen=self.history.maxlen)
        else:
            self.history = []
        print("🗑️  History cleared")

    def show_history(self):
        """Print conversation history."""
        for msg in self.history:
            role = "You " if isinstance(msg, HumanMessage) else "Bot "
            print(f"{role}: {msg.content[:80]}..." if len(msg.content) > 80 else f"{role}: {msg.content}")


# --- Test it ---
bot = StatefulChatbot(
    system_prompt="You are a Python tutor for developers who know JavaScript. Be concise.",
    memory_type="window",
    window_size=10
)

print("Bot:", bot.chat("What's the Python equivalent of Array.map()?"))
print("Bot:", bot.chat("And what about Array.filter()?"))
print("Bot:", bot.chat("Can I chain them like in JS?"))

print("\n📜 Conversation history:")
bot.show_history()

Bot: The Python equivalent of Array.map() is the map() function.
Bot: The Python equivalent of Array.filter() is the filter() function.
Bot: Yes, you can chain map() and filter() in Python using list comprehensions.

📜 Conversation history:
You : What's the Python equivalent of Array.map()?
Bot : The Python equivalent of Array.map() is the map() function.
You : And what about Array.filter()?
Bot : The Python equivalent of Array.filter() is the filter() function.
You : Can I chain them like in JS?
Bot : Yes, you can chain map() and filter() in Python using list comprehensions.


## 🎯 Day 2 Exercises

### Exercise 1: Multi-step Research Chain
Build a sequential chain that:
1. Takes a technology name as input
2. Generates 3 key questions about it
3. Answers all 3 questions
4. Produces a concise summary

### Exercise 2: Persona Chatbot
Create a `StatefulChatbot` that acts as a specific persona (e.g. a senior engineer doing a code review). Test that it stays in character across multiple turns.

### Exercise 3: Token Counter
Extend `StatefulChatbot` to track approximate token usage per message (`len(text.split()) * 1.3` is a rough estimate). Print a warning when total tokens exceed 2000.

In [ ]:
# Exercise 1: Your code here

In [ ]:
# Exercise 2: Your code here

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from collections import deque
from typing import Literal

class StatefulChatbot:
    """
    A chatbot that remembers conversation history.
    
    memory_type: 'buffer' | 'window'
      - 'buffer': keeps all messages (like an ever-growing array)
      - 'window': keeps last k messages (like a circular buffer)
    """
    token_limit = 2000  # Example token limit for context
    def __init__(
        self,
        system_prompt: str = "You are a helpful assistant.",
        memory_type: Literal["buffer", "window"] = "window",
        window_size: int = 10,
    ):
        self.system_prompt = system_prompt
        self.memory_type = memory_type

        # Choose memory strategy
        if memory_type == "window":
            self.history = deque(maxlen=window_size)
        else:
            self.history = []

        # Build the prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])

        self.chain = self.prompt | llm | parser

    def _estimate_tokens(self):
        history_tokens = sum(len(msg.content.split()) for msg in self.history) * 1.3
        system_tokens = len(self.system_prompt.split()) * 1.3
        return int(history_tokens + system_tokens)


    def chat(self, user_input: str) -> str:
        """Send a message and get a response."""
        # Check token limit
        user_tokens = int(len(user_input.split())*1.3)
        total = self._estimate_tokens() + user_tokens
        print(f"total tokens used {total}")
        if total > self.token_limit:
            print('Exceeded token limit of 2000')

        response = self.chain.invoke({
            "history": list(self.history),
            "input": user_input
        })

        # Update history
        self.history.append(HumanMessage(content=user_input))
        self.history.append(AIMessage(content=response))

        return response

    def reset(self):
        """Clear conversation history."""
        if self.memory_type == "window":
            self.history = deque(maxlen=self.history.maxlen)
        else:
            self.history = []
        print("🗑️  History cleared")

    def show_history(self):
        """Print conversation history."""
        for msg in self.history:
            role = "You " if isinstance(msg, HumanMessage) else "Bot "
            print(f"{role}: {msg.content[:80]}..." if len(msg.content) > 80 else f"{role}: {msg.content}")


# --- Test it ---
bot = StatefulChatbot(
    system_prompt="You are a Python tutor for developers who know JavaScript. Be concise.",
    memory_type="window",
    window_size=10
)

print("Bot:", bot.chat("What's the Python equivalent of Array.map()?"))
print("Bot:", bot.chat("And what about Array.filter()?"))
print("Bot:", bot.chat("Can I chain them like in JS?"))

print("\n📜 Conversation history:")
bot.show_history()


total tokens used 22
Bot: In Python, you can use list comprehension or the map() function to achieve similar functionality to Array.map() in JavaScript.
total tokens used 53
Bot: In Python, you can use list comprehension or the filter() function to achieve similar functionality to Array.filter() in JavaScript.
total tokens used 87
Bot: Yes, you can chain list comprehensions or function calls in Python like you would chain Array.map() and Array.filter() in JavaScript.

📜 Conversation history:
You : What's the Python equivalent of Array.map()?
Bot : In Python, you can use list comprehension or the map() function to achieve simil...
You : And what about Array.filter()?
Bot : In Python, you can use list comprehension or the filter() function to achieve si...
You : Can I chain them like in JS?
Bot : Yes, you can chain list comprehensions or function calls in Python like you woul...


## 📝 Day 2 Summary

**What you learned:**
- ✅ Sequential chains — pipe output of one step into the next
- ✅ Parallel chains — run multiple chains simultaneously (`RunnableParallel`)
- ✅ Router chains — conditional logic to pick the right chain
- ✅ `MessagesPlaceholder` — inject dynamic message lists into prompts
- ✅ Three memory strategies: buffer, window, summary
- ✅ Built a reusable `StatefulChatbot` class

**Key concepts:**
- **`StrOutputParser`**: Extracts plain string from LLM response
- **`RunnablePassthrough`**: Passes input through unchanged (useful in parallel chains)
- **`RunnableLambda`**: Wraps any Python function as a chain step
- **`MessagesPlaceholder`**: The key to injecting chat history into a prompt template
- **`deque(maxlen=k)`**: Python's built-in circular buffer — perfect for sliding window memory

**Next up (Day 3):**
- Vector embeddings
- Vector databases (ChromaDB)
- RAG — Retrieval Augmented Generation

Great work! 🎉